# Chapter 5: Sentiment analysis with the Perceptron algorithm

## Importing packages

In [1]:
from matplotlib import pyplot as plt
import numpy
import tqdm

## Plotting functions


We've extended the functionality of our previous plotting functions to include more arguments. As before, we don't use `plt.show()` because there are times when combine these.

In [3]:
# Helpers (Plotting) =========================================

def plot_scatter(x_iterable, y_iterable, x_label = "", y_label = "",  legend = None, **kwargs):
    x_array = numpy.array(x_iterable)
    y_array = numpy.array(y_iterable)
    plt.xlabel(x_label)
    plt.xlabel(y_label)
    if legend is not None:
        plt.legend(legend)
    plt.scatter(x_array, y_array, **kwargs)
        
def draw_line(slope, y_intercept, starting=0, ending=8, **kwargs):
    x = numpy.linspace(starting, ending, 1000)
    plt.plot(x, y_intercept + slope*x, **kwargs)
    


## Coding the perceptron trick

The perceptron algorithm adjusts the line (plane) up or down until the two sets are properly classified. We'll need a few helper functions to score our line (plane) and adjust.

In [5]:
# Helpers (Perceptron) =======================================

def calculate_score(array_feature, array_weights, bias):
    """
    Utilizes the dot function because numpy allows
    vector.dot(scalar) operations
    """
    return array_feature.dot(array_weights) + bias

def step(scalar):
    if scalar >= 0:
        return 1
    else:
        return 0

def prediction(array_feature, array_weights, bias):
    score = calculate_score(array_feature, array_weights, bias)
    return step(score)

def calculate_error(array_feature, array_weights, bias, label):
    """ Correct predictions should have no effect on our adjustment score """
    pred = prediction(array_feature, array_weights, bias)
    if pred == label:
        return 0
    else:
        score = calculate_score(array_feature, array_weights, bias)
        return numpy.abs(score)


# Helpers (Metrics) ==========================================

def calculate_mean_perceptron_error(array_features, array_weights, bias, array_labels):
    """
    Mean error in this case measures how well the entire line (plane) splits the data
    The lower, the better.
    """
    assert array_features.shape[0] == array_labels.shape[0]

    total_error = 0
    for feature, label in zip(array_features, array_labels):
        total_error += calculate_error(feature, array_weights, bias, label)
    return total_error/array_features.shape[0]

# Model ======================================================

def perceptron_trick(array_feature, array_weights, bias, label, learning_rate = 0.01):
    """
    Perceptron trick v1.
    Updates the weights, bias by the learning rate

    If a point is misclassified above the line:
        new weights = old weights - learning_rate * feature
        bias -= learning rate

    If a point is misclassified below the line:
        new weights = old weights + learning_rate * feature
        bias += learning rate
    """

    pred = prediction(array_feature, array_weights, bias)
    if pred == label:
        return array_weights, bias
    else:
        if label==1 and pred==0:
            array_weights = numpy.add(
                array_weights, array_feature*learning_rate)
            bias += learning_rate
        elif label==0 and pred==1:
            array_weights = numpy.subtract(
                array_weights, array_feature*learning_rate)
            bias -= learning_rate

    return array_weights, bias

def perceptron_trick(array_feature, array_weights, bias, label, learning_rate = 0.01):
    """
    Perceptron trick v2.
    Shorter version of the perceptron trick taking full advantage of the fact:
        new weights = old weights + learning_rate * (label – prediction) * feature
        bias += learning_rate * (label – prediction)
    """

    pred = prediction(array_feature, array_weights, bias)
    array_weights = numpy.add(
        array_weights, (label-pred)*array_feature*learning_rate
        )
    bias += (label-pred)*learning_rate

    return array_weights, bias

## Running the full perceptron algorithm

We'll continually adjust our line until we reach convergence (or exhaust our iterations).

In [7]:
def perceptron_algorithm(array_features, array_labels, learning_rate = 0.01, num_epochs = 200):
    """
    Loop breaks when converges or if num_epochs is reached

    Stores the best weights and bias in case of non-convergence
    """
    assert array_features.shape[0] == array_labels.shape[0]

    array_weights = numpy.ones(shape = array_features.shape[1])
    bias = 0.0
    best_weights = None
    best_bias = None

    # base case
    count = 0
    error = calculate_mean_perceptron_error(
        array_features, array_weights, bias, array_labels)
    iter_errors = [error]

    progress_bar = tqdm.tqdm(total = num_epochs)
    while (error >= 1e-16) and (count <= num_epochs):

        error = calculate_mean_perceptron_error(
            array_features, array_weights, bias, array_labels)

        # Identifies best weights
        if error < iter_errors[-1]:
            best_weights = array_weights
            best_bias = bias
        iter_errors.append(error)

        # Updates weights & bias
        index = numpy.random.randint(0, array_features.shape[0] - 1)
        array_weights, bias = perceptron_trick(
            array_features[index], 
            array_weights, 
            bias, 
            array_labels[index],
            learning_rate)

        count +=1

    progress_bar.close()

    # Plotting error
    plot_scatter(range(len(iter_errors)), iter_errors)
    plt.title("Mean Perception Error per Iteration")
    plt.show()

    return best_weights, best_bias